### Using LLamaIndex to Evaluate Correctness

In [1]:
import os
from llama_index.core.evaluation import CorrectnessEvaluator
from llama_index.llms.openai import OpenAI
import asyncio
import pandas as pd
import nest_asyncio
from tqdm import tqdm
import time

llm = OpenAI("gpt-4o")
evaluator = CorrectnessEvaluator(llm=llm)

In [3]:
filtered_df = pd.read_pickle('filtered_df.pkl')
filtered_df

,pair,subj_name,obj_name,pmid,pmid_text,source,first_year,label,context_pmids,path,llm_response_bl,jaccard_index_score_bl
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,1,"{('C3658706', 'C0278579'): ['34345644', '35031...","[C3658706, C0278579, C4724800]",To describe an indirect relationship between p...,0.042857
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,1,"{('C3658706', 'C4704928'): ['29500171', '28946...","[C3658706, C4704928, C0278579, C4724800]","Based on the provided abstract, an indirect re...",0.032258
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,1,"{('C3658706', 'C4704928'): ['33983696', '34956...","[C3658706, C4704928, C4724800]",An indirect relationship between pembrolizumab...,0.031250
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,0,"{('C3658706', 'C1367202'): ['27764796', '28881...","[C3658706, C1367202, C4288736, C4724800]",An indirect relationship between pembrolizumab...,0.029851
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,0,"{('C3658706', 'C2935436'): ['29881714', '34829...","[C3658706, C2935436, C4288736, C4724800]",An indirect relationship between pembrolizumab...,0.030769
385,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,Tisotumab Vedotin in Combination With Carbopla...,title_cooc,2022,0,"{('C3658706', 'C2982078'): ['30859062'], ('C29...","[C3658706, C2982078, C4288736, C4724800]",An indirect relationship between pembrolizumab...,0.029412
383,C3467876 C4524315,larotrectinib,dabrafenib,35955671,The Role of Histology-Agnostic Drugs in the Tr...,sr_pred_sent_2,2022,1,"{('C3467876', 'C0265325'): ['26141621'], ('C02...","[C3467876, C0265325, C4524315]",An indirect relationship between larotrectinib...,0.028169
383,C3467876 C4524315,larotrectinib,dabrafenib,35955671,The Role of Histology-Agnostic Drugs in the Tr...,sr_pred_sent_2,2022,1,"{('C3467876', 'C2697961'): ['32774277'], ('C26...","[C3467876, C2697961, C0265325, C4524315]",An indirect relationship between larotrectinib...,0.041096
383,C3467876 C4524315,larotrectinib,dabrafenib,35955671,The Role of Histology-Agnostic Drugs in the Tr...,sr_pred_sent_2,2022,0,"{('C3467876', 'C1415138'): ['23432625'], ('C14...","[C3467876, C1415138, C0169658, C4524315]",An indirect relationship between larotrectinib...,0.027778
383,C3467876 C4524315,larotrectinib,dabrafenib,35955671,The Role of Histology-Agnostic Drugs in the Tr...,sr_pred_sent_2,2022,0,"{('C3467876', 'C4519250'): ['33991955'], ('C45...","[C3467876, C4519250, C3853694, C4524315]",An indirect relationship between larotrectinib...,0.028169


### Batched Correctness Evaluator:

In [18]:
nest_asyncio.apply()
tqdm.pandas()

async def evaluate_responses_batch(queries, responses, references):
    results = []
    for query, response, reference in zip(queries, responses, references):
        try:
            result = await evaluator.aevaluate(
                query=query,
                response=response,
                reference=reference
            )
            results.append(result)
        except Exception as e:
            print(f"An error occurred: {e}")
            results.append(None) 
    return results

async def synchronous_batch_evaluate(batch_rows):
    queries = []
    responses = []
    references = []
    
    for _, row in batch_rows.iterrows():
        source = row['subj_name']
        target = row['obj_name']
        query = f"How would you describe an indirect relationship between {source} and {target} given the following scientific abstracts as contexts?"
        response = row['llm_response_bl']
        reference = row['pmid_text']
        queries.append(query)
        responses.append(response)
        references.append(reference)
    
    results = await evaluate_responses_batch(queries, responses, references)
    batch_results = pd.DataFrame(columns=['EvalCorrectness_score_bl', 'EvalCorrectness_feedback_bl'])
    
    for i, result in enumerate(results):
        if result:
            batch_results.loc[batch_rows.index[i]] = [result.score, result.feedback]
        else:
            batch_results.loc[batch_rows.index[i]] = [None, 'Error: Could not evaluate']
    
    return batch_results

async def process_batches(df, batch_size):
    all_results = []
    for start in tqdm(range(0, len(df), batch_size), desc="Processing Batches"):
        end = min(start + batch_size, len(df))
        batch_rows = df.iloc[start:end]
        batch_results = await synchronous_batch_evaluate(batch_rows)
        all_results.append(batch_results)
    
    combined_results = pd.concat(all_results)
    combined_results = combined_results.reindex(df.index)
    return combined_results

batch_size = 5
filtered_df = filtered_df.reset_index(drop=True)

filtered_df[['EvalCorrectness_score_bl', 'EvalCorrectness_feedback_bl']] = await process_batches(filtered_df, batch_size)
filtered_df

Processing Batches: 100%|████████████████████████████████████████████████████| 11/11 [01:21<00:00,  7.37s/it]


,pair,subj_name,obj_name,pmid,pmid_text,source,first_year,label,context_pmids,path,llm_response_bl,jaccard_index_score_bl,EvalCorrectness_score_bl,EvalCorrectness_feedback_bl
0,C3658706 C4724800,pembrolizumab,tisotumab vedotin,37651655,"Tisotumab Vedotin in Combination With Carboplatin, Pembrolizumab, or Bevacizumab in Recurrent or Metastatic Cervical Cancer: Results From the innovaTV 205/GOG-3024/ENGOT-cx8 Study. Tissue factor is highly expressed in cervical carcinoma and can be targeted by tisotumab vedotin (TV), an antibody-drug conjugate. This phase Ib/II study evaluated TV in combination with bevacizumab, pembrolizumab, or carboplatin for recurrent or metastatic cervical cancer (r/mCC). This open-label, multicenter study (ClinicalTrials.gov identifier: NCT03786081) included dose-escalation arms that assessed dose-limiting toxicities (DLTs) and identified the recommended phase II dose (RP2D) of TV in combination with bevacizumab (arm A), pembrolizumab (arm B), or carboplatin (arm C). The dose-expansion arms evaluated TV antitumor activity and safety at RP2D in combination with carboplatin as first-line (1L) treatment (arm D) or with pembrolizumab as 1L (arm E) or second-/third-line (2L/3L) treatment (arm F). The primary end point of dose expansion was objective response rate (ORR). A total of 142 patients were enrolled. In dose escalation (n = 41), no DLTs were observed; the RP2D was TV 2 mg/kg plus bevacizumab 15 mg/kg on day 1 once every 3 weeks, pembrolizumab 200 mg on day 1 once every 3 weeks, or carboplatin AUC 5 on day 1 once every 3 weeks. In dose expansion (n = 101), the ORR was 54.5% (n/N, 18/33; 95% CI, 36.4 to 71.9) with 1L TV + carboplatin (arm D), 40.6% (n/N, 13/32; 95% CI, 23.7 to 59.4) with 1L TV + pembrolizumab (arm E), and 35.3% (12/34; 19.7 to 53.5) with 2L/3L TV + pembrolizumab (arm F). The median duration of response was 8.6 months, not reached, and 14.1 months, in arms D, E, and F, respectively. Grade >=3 adverse events (>=15%) were anemia, diarrhea, nausea, and thrombocytopenia in arm D and anemia in arm F (none >=15%, arm E). TV in combination with bevacizumab, carboplatin, or pembrolizumab demonstrated manageable safety and encouraging antitumor activity in treatment-naive and previously treated r/mCC.",title_cooc,2022,1,"{('C3658706', 'C0278579'): ['34345644', '35031189', '29666026', '32740092'], ('C0278579', 'C4724800'): ['35277279']}","[C3658706, C0278579, C4724800]","To describe an indirect relationship between pembrolizumab and tisotumab vedotin based on the provided scientific abstract, we can focus on the context of their use in treating recurrent cervical cancer and the potential for complementary or sequential therapeutic strategies. Here’s a detailed description:\n\n---\n\n**Indirect Relationship Between Pembrolizumab and Tisotumab Vedotin in Recurrent Cervical Cancer**\n\nPembrolizumab, an anti-PD-1 therapy, has shown promise in treating advanced or recurrent PD-L1 positive cervical cancer, with a subset of patients deriving durable clinical benefits. In a retrospective cohort study of 14 patients with recurrent or metastatic cervical cancer treated with pembrolizumab, the objective response rate was 21%, with a durable clinical benefit rate of 36%. Key factors associated with improved response included metastatic spread limited to the lung and/or lymph nodes and higher tumor mutational burden.\n\nTisotumab vedotin, an antibody-drug conjugate targeting tissue factor, represents another therapeutic option for cervical cancer. While the abstract does not provide specific data on tisotumab vedotin, its mechanism of action and clinical application suggest it could be used in patients who do not respond to or progress after pembrolizumab treatment.\n\nThe indirect relationship between pembrolizumab and tisotumab vedotin can be understood in the context of their complementary roles in the treatment landscape of recurrent cervical cancer. Pembrolizumab may be particularly beneficial for patients

In [20]:
mean_0_ec = filtered_df[filtered_df['label'] == 0]['EvalCorrectness_score_bl'].mean()
mean_1_ec = filtered_df[filtered_df['label'] == 1]['EvalCorrectness_score_bl'].mean()

print(f"Mean EvalCorrectness_score_bl for label 0: {mean_0_ec}")
print(f"Mean EvalCorrectness_score_bl for label 1: {mean_1_ec}")

Mean EvalCorrectness_score_bl for label 0: 2.2666666666666666
Mean EvalCorrectness_score_bl for label 1: 2.9565217391304346


In [21]:
filtered_df.to_pickle('filtered_df_now.pkl')

In [8]:
filtered_df['EvalCorrectness_score_bl'].mean()

2.792452830188679

In [ ]:
filtered_df['EvalCorrectness_score_bl'].hist(bins=20)

In [8]:


nest_asyncio.apply()
tqdm.pandas()

async def evaluate_response(query, response, reference):
    result = await evaluator.aevaluate(
        query=query,
        response=response,
        reference=reference
    )
    return result

def synchronous_evaluate(row):
    source = row['subj_name']
    target = row['obj_name']
    query = f"How would you describe an indirect relationship between {source} and {target} given the following scientific abstracts as contexts?"
    response = row['llm_response_bl']
    reference = row['pmid_text']
    result = asyncio.run(evaluate_response(query, response, reference))
    return pd.Series({'EvalCorrectness_score_bl': result.score, 'EvalCorrectness_feedback_bl': result.feedback})


filtered_df[['EvalCorrectness_score_bl', 'EvalCorrectness_feedback_bl']] = filtered_df.progress_apply(synchronous_evaluate, axis=1)
filtered_df

  0%|                                                                                 | 0/53 [00:00<?, ?it/s]Retrying llama_index.llms.openai.base.OpenAI._achat in 0.49083613639291035 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}.
Retrying llama_index.llms.openai.base.OpenAI._achat in 1.862115841045188 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}.
  2%|█▍                                      

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

### Sanity check: for identical value it is 5

In [11]:
query = "Can you explain the theory of relativity proposed by Albert Einstein in detail?"


reference = """
Certainly! Albert Einstein's theory of relativity consists of two main components: special relativity and general relativity. Special relativity, published in 1905, introduced the concept that the laws of physics are the same for all non-accelerating observers and that the speed of light in a vacuum is a constant, regardless of the motion of the source or observer. It also gave rise to the famous equation E=mc², which relates energy (E) and mass (m).

General relativity, published in 1915, extended these ideas to include the effects of gravity. According to general relativity, gravity is not a force between masses, as described by Newton's theory of gravity, but rather the result of the warping of space and time by mass and energy. Massive objects, such as planets and stars, cause a curvature in spacetime, and smaller objects follow curved paths in response to this curvature. This concept is often illustrated using the analogy of a heavy ball placed on a rubber sheet, causing it to create a depression that other objects (representing smaller masses) naturally move towards.

In essence, general relativity provided a new understanding of gravity, explaining phenomena like the bending of light by gravity (gravitational lensing) and the precession of the orbit of Mercury. It has been confirmed through numerous experiments and observations and has become a fundamental theory in modern physics.
"""

response = """
Certainly! Albert Einstein's theory of relativity consists of two main components: special relativity and general relativity. Special relativity, published in 1905, introduced the concept that the laws of physics are the same for all non-accelerating observers and that the speed of light in a vacuum is a constant, regardless of the motion of the source or observer. It also gave rise to the famous equation E=mc², which relates energy (E) and mass (m).

General relativity, published in 1915, extended these ideas to include the effects of gravity. According to general relativity, gravity is not a force between masses, as described by Newton's theory of gravity, but rather the result of the warping of space and time by mass and energy. Massive objects, such as planets and stars, cause a curvature in spacetime, and smaller objects follow curved paths in response to this curvature. This concept is often illustrated using the analogy of a heavy ball placed on a rubber sheet, causing it to create a depression that other objects (representing smaller masses) naturally move towards.

In essence, general relativity provided a new understanding of gravity, explaining phenomena like the bending of light by gravity (gravitational lensing) and the precession of the orbit of Mercury. It has been confirmed through numerous experiments and observations and has become a fundamental theory in modern physics.
"""

import nest_asyncio
nest_asyncio.apply()
import asyncio

async def evaluate_response(query, response, reference, evaluator):
    result = await evaluator.aevaluate(
        query=query,
        response=response,
        reference=reference  
    )
    return result

result = asyncio.run(evaluate_response(query, response, reference, evaluator))

print(result.score)
print(result.feedback)

5.0
The generated answer is identical to the reference answer and provides a detailed and accurate explanation of Albert Einstein's theory of relativity, covering both special and general relativity. It is relevant, correct, and comprehensive.


### Experiment shuffling words correctnessEvaluator Score:

In [59]:
import random

def shuffle_words(text):
    # Split text into words
    words = text.split()
    # Shuffle the words
    random.shuffle(words)
    # Join the shuffled words back into a string
    shuffled_text = ' '.join(words)
    return shuffled_text

In [31]:
import nest_asyncio
nest_asyncio.apply()
import asyncio

async def evaluate_response(query, response, reference, evaluator):
    result = await evaluator.aevaluate(
        query=query,
        response=response,
        reference=reference  
    )
    return result

result = asyncio.run(evaluate_response(query, response_bl, reference, evaluator))

### Conclusion:
What is the scale? 
what does functionality wise correctness do? It use search_semantic_similarity (i.e. String wise)
- We try injecting at middle of sentence some non-sense(irelavant context) It droped into 3.5.
- when we use whole non-sense, instead just injecting only one sentence at the middle, it drops into 1.
- we use function to randomly shuffle text basically, It droped into 1.